In [1]:
# Import necessary libraries
import h2o
from h2o.automl import H2OAutoML

# Initialize the H2O cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp0zot2ba4
  JVM stdout: /tmp/tmp0zot2ba4/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp0zot2ba4/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,1 month and 20 days
H2O_cluster_name:,H2O_from_python_unknownUser_6s7o46
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [2]:
# Load your data into a H2OFrame
data = h2o.import_file('/kaggle/input/optiver-trading-at-the-close/train.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [3]:
# Specify the response and predictor columns
response_column = 'target'
predictor_columns = data.columns
predictor_columns.remove("target")

In [4]:
# Need to detect best model on reduced dataset due to size
## Determine the ratio for your sample
ratio = 0.01
## Split frame
splits = data.split_frame(ratios = [ratio], seed = 1234)
# Sample is the first split
sample = splits[0]
print("sample size train+val is:",sample.nrows)

# Split the sample into training and validation sets
train_sample, val_sample = sample.split_frame(ratios=[0.8], seed=1)

# Initialize and train the H2OAutoML model
#aml = H2OAutoML(max_models=20, seed=1)
aml = H2OAutoML(max_models=10,include_algos=["DeepLearning"])
aml.train(x=predictor_columns, y=response_column, training_frame=train_sample)

# View the AutoML Leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))

# Predict on the validation set
preds = aml.predict(val_sample)

## Convert the predictions back to a pandas dataframe (optional)
#preds_df = h2o.as_list(preds)

sample size train+val is: 52388
AutoML progress: |
10:07:19.261: _train param, Dropping bad and constant columns: [row_id]

███████████████████████████████████████████████████████████████| (done) 100%
model_id                                                  rmse      mse      mae    rmsle    mean_residual_deviance
DeepLearning_grid_2_AutoML_1_20231013_100719_model_9   9.37335  87.8598  6.36538      nan                   87.8598
DeepLearning_grid_2_AutoML_1_20231013_100719_model_6   9.40139  88.3862  6.39512      nan                   88.3862
DeepLearning_grid_2_AutoML_1_20231013_100719_model_10  9.40817  88.5137  6.40336      nan                   88.5137
DeepLearning_grid_2_AutoML_1_20231013_100719_model_5   9.41159  88.578   6.39014      nan                   88.578
DeepLearning_grid_3_AutoML_1_20231013_100719_model_1   9.41328  88.6098  6.41085      nan                   88.6098
DeepLearning_grid_2_AutoML_1_20231013_100719_model_8   9.41358  88.6155  6.39889      nan               

In [5]:
# Extract best model
best_model = aml.leader

# Save the model
h2o.save_model(model=best_model, path="/kaggle/working/", force=True)

'/kaggle/working/DeepLearning_grid_2_AutoML_1_20231013_100719_model_9'

In [6]:
# Split the data into training and validation sets
train, val = data.split_frame(ratios=[0.8], seed=1)

In [7]:
# Continue training the model on the full data set
best_model.train(x=predictor_columns, y=response_column, training_frame=train)

deeplearning Model Build progress: |

/opt/conda/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: Dropping bad and constant columns: [row_id]
  warnings.warn(mesg["message"], RuntimeWarning)


█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1697191607977_7


Status of Neuron Layers: predicting target, regression, gaussian distribution, Quadratic loss, 43,601 weights/biases, 521.9 KB, 1,688,885 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate            rate_rms              momentum    mean_weight            weight_rms           mean_bias            bias_rms
--  -------  -------  ---------  ---------  ----  ----  -------------------  --------------------  ----------  ---------------------  -------------------  -------------------  -----------------------
    1        15       Input      0.0
    2        200      Rectifier  0.0        0.0   0.0   0.04023311459179968  0.10358932614326477   0.0         0.01667122230324943    0.23255705833435059  0.2487330327300318   0.14056873321533203
    3        200      Rectifier  0.0        0.0   0.0   0.12666274930774235  0.1238192617893219    0.0         -0.03560432966652786   0.07894819974899292  0.8921085568553799   0.029324844479560852
    4        1        Linear                0.0   0.0   0.00108822268930453  0.000696912407875061  0.0         0.0014586979972955306  0.03146591782569885  0.02391505049096071  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 84.43359063043982
RMSE: 9.188775251927746
MAE: 6.298971483844302
RMSLE: NaN
Mean Residual Deviance: 84.43359063043982

Scoring History: 
     timestamp            duration          training_speed    epochs                iterations    samples    training_rmse      training_deviance    training_mae        training_r2
---  -------------------  ----------------  ----------------  --------------------  ------------  ---------  -----------------  -------------------  ------------------  --------------------
     2023-10-13 11:13:57  0.000 sec                           0.0                   0             0.0        nan                nan                  nan                 nan
     2023-10-13 11:14:04  8.782 sec         6670 obs/sec      0.01094206617878834   1             45856.0    9.327523014208628  87.00268558059162    6.394850456339683   0.004750141980551237
     2023-10-13 11:14:11  16.224 sec        9788 obs/sec      0.032712854994954424  3             137093.0   9.278824509276118  86.0965842739432     6.358395970016657   0.01511530704155084
     2023-10-13 11:14:18  23.256 sec        11023 obs/sec     0.05447075844009698   5             228276.0   9.259753625306493  85.74303720137672    6.3575953146633974  0.01915963821853539
     2023-10-13 11:14:24  29.436 sec        12040 obs/sec     0.07631456435872969   7             319819.0   9.213636432272136  84.8910963060924     6.318508548930096   0.02890524606287348
     2023-10-13 11:14:31  36.529 sec        12341 obs/sec     0.09819893533428828   9             411532.0   9.235034515807747  85.28586250816042    6.342043971729762   0.024389396880319825
     2023-10-13 11:14:38  42.941 sec        12749 obs/sec     0.12001386847710901   11            502954.0   9.226505218323947  85.12839854375902    6.323186151134474   0.026190674475004005
     2023-10-13 11:14:44  48.817 sec        13197 obs/sec     0.1417259572697235    13            593945.0   9.307023203817174  86.6206809163913     6.389819708779129   0.009120008097568522
     2023-10-13 11:14:50  55.028 sec        13461 obs/sec     0.16348553104073948   15            685135.0   9.217944628276445  84.97050316997057    6.3244227461025195  0.027996887091272238
     2023-10-13 11:14:56  1 min  1.451 sec  13628 obs/sec     0.1853799239715386    17            776890.0   9.204867278504128  84.72958161487601    6.309903506445144   0.030752861138528265
---  ---                  ---               ---               ---                   ---           ---        ---                ---                  ---                 ---
     2023-10-13 11:15:09  1 min 14.573 sec  13812 ob

In [8]:
# Save the model
h2o.save_model(model=best_model, path="/kaggle/working/", force=True)

'/kaggle/working/DeepLearning_model_python_1697191607977_7'

In [9]:
# Predict on the validation set
preds = best_model.predict(val)

## Convert the predictions back to a pandas dataframe (optional)
#preds_df = h2o.as_list(preds)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
